## はじめに

このノートブックでは自己教師あり対照学習(Self-Supervised Contrastive Learning)を用いて`palette.csv`(`color.csv`にも適用可能)を固定長のベクトルに変換する手法を紹介します。ここで紹介する内容はGPUがないと時間が大幅にかかってしまうため若干手が出しづらいかもしれないことをご承知おきください。

なお、今回の内容はどちらかというと興味ドリブンでやってみた系のお話なのでハードルが高い割にスコアにはあまり響かないかもしれません。一応、私の手元の実験ではCVが1.014 → 1.006、LBが0.9876 → 0.9847とCV,LBの両方に寄与しました。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/second_take

/content/drive/MyDrive/Colab Notebooks/second_take


In [ ]:
!pip install catalyst

     |████████████████████████████████| 460kB 5.6MB/s 
     |████████████████████████████████| 317kB 7.9MB/s 


In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint

# checkpoint = ModelCheckpoint(filepath = './model_temp/model_001.h5',
#                              monitor='loss',
#                              save_best_only=True,
#                              save_weight_only=False,
#                              mode='min',
#                              save_freq=1)

In [ ]:
!pip install -U git+https://github.com/albu/albumentations > /dev/null 

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-dzuuy0ly


In [ ]:
import os
import random

import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torchdata
import umap

from pathlib import Path

from albumentations.pytorch import ToTensorV2
# from albumentations.pytorch import ToTensor
from catalyst.dl import SupervisedRunner, Runner
from catalyst.core import Callback, CallbackOrder, IRunner
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
sns.set_context("talk")
plt.style.use("ggplot")

## モチベーション

今回与えられたデータには`palette.csv`や`color.csv`のように作品中の配色の比率を示したデータも与えられており、[パレットの可視化](https://www.guruguru.science/competitions/16/discussions/0cf48a1f-59fd-45b1-880a-cf9fc54d6912/)などでも議論されているように色のバリエーションや鮮やかさなどは予測対象の`likes`とも相関が高そうです。しかしながら一つの作品(`object_id`)に与えられている色の種類はさまざまでこれをうまく固定長の特徴表現に直すのは人手ではなかなか難しそうです。

In [ ]:
DATADIR = Path("./input/")

palette = pd.read_csv(DATADIR / "palette.csv")
palette.head()

,ratio,color_r,color_g,color_b,object_id
0,0.013781,40,4,0,000405d9a5e3f49fc49d
1,0.040509,221,189,129,000405d9a5e3f49fc49d
2,0.036344,207,175,117,000405d9a5e3f49fc49d
3,0.033316,230,197,129,000405d9a5e3f49fc49d
4,0.039600,194,161,106,000405d9a5e3f49fc49d


これを踏まえ、[`palette`をまず画像化し](https://www.guruguru.science/competitions/16/discussions/eb65c133-a5e9-43b9-b046-8b6a7184ad5e/)、[学習済みCNNの重みをファインチューンする形で今回のタスクに利用](https://www.guruguru.science/competitions/16/discussions/88babff4-5383-4da4-9496-10b8f1ccad30/)したり、[学習済みCNNを用いて特徴ベクトルに変換](https://www.guruguru.science/competitions/16/discussions/cccaada9-b29b-46d6-93d9-01d992362ce1/)してLightGBMなどで利用する、などの取組が既に紹介されています。

しかし、このやり方ではまず`palette.csv`をグラデーションのある画像のように変換する、という手順によって色という情報の他に、「縦に線が入っている」・「左から右にかけて徐々に領域が狭くなる」といった人間が後から処理の都合で付け加えた情報も入るためモデルがこれらの実際は意味のない情報も含めてベクトル化してしまっている可能性もあります。

実際には色の比率の情報のみが含まれるため、その配置に関してはランダムでも全く問題はないはずです。実際に各色を`palette.csv`に指示されている比率に従ってランダムに配置した場合はどうなるかみてみましょう。

まず少しトリッキーですが、`palette`の`ratio`をパーセント表示に直した上で四捨五入した整数に直しておきます。その上で`object_id`で集約した時に足し合わせてちょうど100になるようにします。これは後ほど画像化するときに和がちょうど100であると都合がいいからです。

In [ ]:
# パーセント表示に直して四捨五入
palette["ratio_int"] = palette["ratio"].map(lambda x: int(np.round(10000 * x)))

# `object_id`で集約してratio_intを足し合わせると100を超えたり100に満たない場合がある
palette.groupby("object_id")["ratio_int"].sum()

object_id
000405d9a5e3f49fc49d     9998
001020bd00b149970f78     9998
0011d6be41612ec9eae3    10001
0012765f7a97ccc3e9e9    10002
00133be3ff222c9b74b0    10001
                        ...  
fff4bbb55fd7702d294e    10000
fffbe07b997bec00e203    10000
fffd1675758205748d7f    10001
fffd43b134ba7197d890     9998
ffff22ea12d7f99cff31     9999
Name: ratio_int, Length: 23995, dtype: int64

In [ ]:
# `object_id`で集約した時に足し合わせてちょうど100になるようにする
palette_group_dfs = []
for _, df in tqdm(palette.groupby("object_id"),
                  total=palette["object_id"].nunique()):
    # 足し合わせた和が100を超過する場合
    if df["ratio_int"].sum() > 10000:
        n_excess = df["ratio_int"].sum() - 10000
        # ちょっと雑だが一番比率が多い色の割合を減らすことで和を100に揃える
        max_ratio_int_idx = df["ratio_int"].idxmax()
        df.loc[max_ratio_int_idx, "ratio_int"] -= n_excess
    elif df["ratio_int"].sum() < 10000:
        n_lack = 10000 - df["ratio_int"].sum()
        max_ratio_int_idx = df["ratio_int"].idxmax()
        df.loc[max_ratio_int_idx, "ratio_int"] += n_lack
    else:
        pass
    palette_group_dfs.append(df)
    
new_palette = pd.concat(palette_group_dfs, axis=0).reset_index(drop=True)

In [ ]:
# `object_id`で集約してratio_intを足し合わせるとちょうど100になる
new_palette.groupby("object_id")["ratio_int"].sum()

object_id
000405d9a5e3f49fc49d    10000
001020bd00b149970f78    10000
0011d6be41612ec9eae3    10000
0012765f7a97ccc3e9e9    10000
00133be3ff222c9b74b0    10000
                        ...  
fff4bbb55fd7702d294e    10000
fffbe07b997bec00e203    10000
fffd1675758205748d7f    10000
fffd43b134ba7197d890    10000
ffff22ea12d7f99cff31    10000
Name: ratio_int, Length: 23995, dtype: int64

さて、この`new_palette`を使って10x10の画像で各ピクセルが指示された比率だけ指示された色になったような画像をランダムに生成してみます。

In [ ]:
def _create_random_image(sample: pd.DataFrame) -> np.ndarray:
    """
    配置はランダムで色の比率がsampleで指示された値になるようにした
    10x10の画像を生成する
    """
    # まず一次元で定義しておく
    image = np.zeros((10000, 3), dtype=np.uint8)
    # sampleの頭から1行ずつその行の色をその行のratio_int分だけコピーして画像を埋める
    head = 0
    for i, row in sample.iterrows():
        # sampleの行に書かれた色
        patch = np.array([[row.color_r, row.color_g, row.color_b]], dtype=np.uint8)
        # sampleの行に書かれたratio_int分だけコピーする
        patch = np.tile(patch, row.ratio_int).reshape(row.ratio_int, -1)
        # 画像を上の手順で出した色で埋める
        image[head:head + row.ratio_int, :] = patch
        head += row.ratio_int
    # 乱数で順番をランダム化する
    indices = np.random.permutation(np.arange(10000))
    image = image[indices, :].reshape(100, 100, 3)
    return image

この関数で何枚か画像を生成してみましょう。

## 一旦copastaの関数を記述

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/second_take/src

/content/drive/MyDrive/Colab Notebooks/second_take/src


In [ ]:
#===========================================================
# Config
#===========================================================

#===========================================================
# Config
#===========================================================
import yaml

with open('./config.yaml') as file:
    config = yaml.safe_load(file.read())

config

df_path_dict = {
    'train': config['input_dir_root_jn']+'train.csv',
    'test': config['input_dir_root_jn']+'test.csv',
    'sample_submission': config['input_dir_root_jn']+'sample_submission.csv',
    'folds': config['input_dir_jn']+'folds.csv',
}

In [ ]:
#===========================================================
# Library
#===========================================================

import gc
import itertools
import json
import os
import random
import sys
import time
import warnings
from collections import Counter, defaultdict
from contextlib import contextmanager
from functools import partial
from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

warnings.filterwarnings("ignore")

import builtins
import types

import lightgbm as lgb
import matplotlib.pyplot as plt
#import MeCab
# # import mojimoji
# import neologdn
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import torch
import xgboost as xgb
# from catboost import CatBoostClassifier, CatBoostRegressor
from gensim.models.word2vec import Word2Vec
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import NMF, PCA, TruncatedSVD
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer,
                                             _document_frequency)
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import (GroupKFold, GroupShuffleSplit, KFold,
                                     StratifiedKFold)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.validation import check_is_fitted
from tqdm.notebook import tqdm
from PIL import ImageColor



from pathlib import Path

from gensim.models import word2vec, KeyedVectors
from tqdm import tqdm

# import texthero as hero
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

import nltk

nltk.download('stopwords')
os.listdir(os.path.expanduser('~/nltk_data/corpora/stopwords/'))

class AbstractBaseBlock:
    def fit(self, input_df: pd.DataFrame, y=None):
        return self.transform(input_df)
    
    def transform(self, input_df: pd.DataFrame) -> pd.DataFrame:
        raise NotImplementedError()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#===========================================================
# Utils
#===========================================================

def seed_everything(seed=1996):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


@contextmanager
def timer(name):
    t0 = time.time()
    logger.info(f'[{name}] start')
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')
    logger.info('')


def get_logger(filename='log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log", mode='w')
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = get_logger(config['output_dir_jn']+config['fname_log_pp'])

def load_df(path, df_name, config):
    if path.split('.')[-1]=='csv':
        if config['debug']:
            df = pd.read_csv(path, nrows=1000)
        else:
            df = pd.read_csv(path)
    elif path.split('.')[-1]=='pkl':
        df = pd.read_pickle(path)
    logger.info(f"{df_name} shape / {df.shape} ")
    return df

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        logger.info('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df



def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )




# https://github.com/nyk510/vivid/blob/master/vivid/utils.py

def decorate(s: str, decoration=None):
    if decoration is None:
        decoration = '★' * 20
        
    return ' '.join([decoration, str(s), decoration])

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        if self.verbose is None:
            return
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)
    

def run_blocks(input_df, blocks, y=None, test=False):
    out_df = pd.DataFrame()
    
    print(decorate('start run blocks...'))

    with Timer(prefix='run test={}'.format(test)):
        for block in feature_blocks:
            with Timer(prefix='\t- {}'.format(str(block))):
                if not test:
                    out_i = block.fit(input_df, y=y)
                else:
                    out_i = block.transform(input_df)

            assert len(input_df) == len(out_i), block
            name = block.__class__.__name__
            out_df = pd.concat([out_df, out_i.add_suffix(f'_{name}')], axis=1)
        
    return out_df

In [ ]:
with timer('Data Loading'):
    train = load_df(path=df_path_dict['train'], df_name='train', config=config)
    test = load_df(path=df_path_dict['test'], df_name='test', config=config)
    sample_submission = load_df(path=df_path_dict['sample_submission'], df_name='sample_submission', config=config)
    folds = load_df(path=df_path_dict['folds'], df_name='folds', config=config)
    gc.collect()

[Data Loading] start
train shape / (12026, 19) 
test shape / (12008, 18) 
sample_submission shape / (12008, 1) 
folds shape / (12026, 3) 
[Data Loading] done in 2 s



In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/second_take

/content/drive/MyDrive/Colab Notebooks/second_take


In [ ]:
train = train[['object_id', 'likes']]
test = test[['object_id']]

# データ作成

In [ ]:
# unique_object_ids = train["object_id"].unique()
# #unique_object_ids = test["object_id"].unique()
# unique_palette_obj = new_palette["object_id"].unique()
# N_IMAGES_FOR_OBJ_ID = 5
# N_OBJ_IDS = len(unique_object_ids)
# train_X = []

# # fig, axes = plt.subplots(nrows=N_OBJ_IDS, ncols=N_IMAGES_FOR_OBJ_ID, figsize=(25, 25))
# for i in tqdm(range(N_OBJ_IDS)):
#     obj_id = unique_object_ids[i]
#     palette_obj_id = new_palette.query(f"object_id == '{obj_id}'")
#     #print(obj_id)
#     if obj_id not in unique_palette_obj:
#       print(_create_random_image(palette_obj_id))
#       print(palette_obj_id)
#       break

In [ ]:
unique_object_ids = train["object_id"].unique()
N_IMAGES_FOR_OBJ_ID = 1
N_OBJ_IDS = len(unique_object_ids)
unique_palette_obj = new_palette["object_id"].unique()
train_X = []

# fig, axes = plt.subplots(nrows=N_OBJ_IDS, ncols=N_IMAGES_FOR_OBJ_ID, figsize=(25, 25))
for i in tqdm(range(N_OBJ_IDS)):
    obj_id = unique_object_ids[i]
    if obj_id not in unique_palette_obj:
      continue

    palette_obj_id = new_palette.query(f"object_id == '{obj_id}'")
    #axes[i, 0].set_ylabel(obj_id)
    for j in range(N_IMAGES_FOR_OBJ_ID):
        generated = _create_random_image(palette_obj_id)
        train_X.append(generated)
        #axes[i, j].imshow(generated)
        #axes[i, j].grid(False)
        #axes[i, j].set_title(f"Generated Image {j}")
        
#plt.tight_layout()
#plt.show()

100%|██████████| 12026/12026 [03:48<00:00, 52.70it/s]


In [ ]:
np.array(train_X).shape

(12007, 100, 100, 3)

In [ ]:
import os, zipfile, io, re
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception
from keras.models import Model, load_model
from keras.layers.core import Dense
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_squared_log_error

In [ ]:
Y = []
for i, like in tqdm(zip(range(N_OBJ_IDS), train['likes'])):
  obj_id = unique_object_ids[i]
  if obj_id not in unique_palette_obj:
      continue
  Y.append([like]*N_IMAGES_FOR_OBJ_ID)
Y = np.ravel(Y)

12026it [00:06, 1931.07it/s]


In [ ]:
X = train_X.copy()
X = np.array(X)
Y = np.array(Y)

In [ ]:
# trainデータとtestデータに分割
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    Y,
    random_state = 0,
    test_size = 0.2
)
del X,Y
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape) 
X_train = X_train.astype('float32') / 255
X_valid = X_valid.astype('float32') / 255
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')

(9605, 100, 100, 3) (9605,) (2402, 100, 100, 3) (2402,)


## 学習準備

In [ ]:
base_model = Xception(
    include_top = False,
    weights = "imagenet",
    input_shape = None
)

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1)(x)

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 0,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    vertical_flip = False
)

In [ ]:
# EarlyStopping
early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 10,
    verbose = 1
)

# ModelCheckpoint
weights_dir = './weights/'
if os.path.exists(weights_dir) == False:os.mkdir(weights_dir)
model_checkpoint = ModelCheckpoint(
    weights_dir + "val_loss{val_loss:.3f}.hdf5",
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
    period = 3
)

# reduce learning rate
reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.1,
    patience = 3,
    verbose = 1
)

# log for TensorBoard
logging = TensorBoard(log_dir = "log/")

In [ ]:
# RMSE
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1)) 

In [ ]:
# RMSLE
from keras import backend as K
def root_mean_squared_log_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))

In [ ]:
#===========================================================
# Metrics
#===========================================================

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))
    
def get_score(y_true, y_pred):
    score = rmsle(y_true, y_pred)
    return score

def custom_eval(preds, data):
    y_true = data.get_label()
    y_pred = np.where(preds > 0.5, 1, 0)
    metric = np.mean(y_true == y_pred)
    return 'accuracy', metric, True

In [ ]:
# ネットワーク定義
model = Model(inputs = base_model.input, outputs = predictions)

#108層までfreeze
for layer in model.layers[:108]:
    layer.trainable = False

    # Batch Normalizationのfreeze解除
    if layer.name.startswith('batch_normalization'):
        layer.trainable = True
    if layer.name.endswith('bn'):
        layer.trainable = True

#109層以降、学習させる
for layer in model.layers[108:]:
    layer.trainable = True

# layer.trainableの設定後にcompile
model.compile(
    optimizer = Adam(),
    loss = root_mean_squared_log_error,
)

## 学習開始

In [ ]:
#%%time
hist = model.fit_generator(
    datagen.flow(X_train, y_train, batch_size = 32),
    steps_per_epoch = X_train.shape[0] // 32,
    epochs = 50,
    validation_data = (X_valid, y_valid),
    callbacks = [early_stopping, reduce_lr],
    shuffle = True,
    verbose = 1
)

Epoch 1/50
300/300 [==============================] - 67s 108ms/step - loss: 1.5251 - val_loss: 1.3340
Epoch 2/50
300/300 [==============================] - 31s 102ms/step - loss: 1.3761 - val_loss: 1.4701
Epoch 3/50
300/300 [==============================] - 31s 102ms/step - loss: 1.3901 - val_loss: 1.3647
Epoch 4/50
300/300 [==============================] - 30s 101ms/step - loss: 1.3244 - val_loss: 1.4795

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/50
300/300 [==============================] - 32s 106ms/step - loss: 1.3481 - val_loss: 1.3075
Epoch 6/50
300/300 [==============================] - 32s 106ms/step - loss: 1.2998 - val_loss: 1.3062
Epoch 7/50
300/300 [==============================] - 32s 107ms/step - loss: 1.3202 - val_loss: 1.3121
Epoch 8/50
300/300 [==============================] - 32s 106ms/step - loss: 1.2945 - val_loss: 1.2960
Epoch 9/50
300/300 [==============================] - 32s 106ms/step - loss: 1.3053 - val_loss

In [ ]:
plt.figure(figsize=(18,6))

# loss
plt.subplot(1, 2, 1)
plt.plot(hist.history["loss"], label="loss", marker="o")
plt.plot(hist.history["val_loss"], label="val_loss", marker="o")
#plt.yticks(np.arange())
#plt.xticks(np.arange())
plt.ylabel("loss")
plt.xlabel("epoch")
plt.title("")
plt.legend(loc="best")
plt.grid(color='gray', alpha=0.2)

plt.show()

## testのデータ作成

In [ ]:
unique_object_ids = test["object_id"].unique()
N_IMAGES_FOR_OBJ_ID = 1
N_OBJ_IDS = len(unique_object_ids)
unique_palette_obj = new_palette["object_id"].unique()
X_test = []

# fig, axes = plt.subplots(nrows=N_OBJ_IDS, ncols=N_IMAGES_FOR_OBJ_ID, figsize=(25, 25))
for i in tqdm(range(N_OBJ_IDS)):
    obj_id = unique_object_ids[i]
    if obj_id not in unique_palette_obj:
      continue

    palette_obj_id = new_palette.query(f"object_id == '{obj_id}'")
    #axes[i, 0].set_ylabel(obj_id)
    for j in range(N_IMAGES_FOR_OBJ_ID):
        generated = _create_random_image(palette_obj_id)
        X_test.append(generated)

100%|██████████| 12008/12008 [04:00<00:00, 49.93it/s]


In [ ]:
X_test = np.array(X_test)
X_test = X_test.astype('float32') / 255

In [ ]:
X_test.shape

(11988, 100, 100, 3)

## 予測

In [ ]:
y_pred = model.predict(X_test, verbose=1)

375/375 [==============================] - 7s 18ms/step


In [ ]:
y_test_pred = np.ravel(y_pred)

In [ ]:
train_X = np.array(train_X)
train_X = train_X.astype('float32') / 255
y_train_pred = model.predict(train_X, verbose=1)
y_train_pred = np.ravel(y_train_pred)

376/376 [==============================] - 7s 18ms/step


In [ ]:
unique_train_ids = train["object_id"].unique()
unique_test_ids = test["object_id"].unique()
N_IMAGES_FOR_OBJ_ID = 1
N_OBJ_IDS = len(unique_train_ids)
unique_palette_obj = new_palette["object_id"].unique()

X_predict = []

# fig, axes = plt.subplots(nrows=N_OBJ_IDS, ncols=N_IMAGES_FOR_OBJ_ID, figsize=(25, 25))
for i in tqdm(range(N_OBJ_IDS)):
    obj_id = unique_train_ids[i]
    if obj_id not in unique_palette_obj:
      continue
    X_predict.append(obj_id)

100%|██████████| 12026/12026 [00:06<00:00, 1886.73it/s]


In [ ]:
unique_train_ids = train["object_id"].unique()
unique_test_ids = test["object_id"].unique()
N_IMAGES_FOR_OBJ_ID = 1
N_OBJ_IDS = len(unique_test_ids)
unique_palette_obj = new_palette["object_id"].unique()

X_test_predict = []

# fig, axes = plt.subplots(nrows=N_OBJ_IDS, ncols=N_IMAGES_FOR_OBJ_ID, figsize=(25, 25))
for i in tqdm(range(N_OBJ_IDS)):
    obj_id = unique_test_ids[i]
    if obj_id not in unique_palette_obj:
      continue
    X_test_predict.append(obj_id)

100%|██████████| 12008/12008 [00:06<00:00, 1894.94it/s]


In [ ]:
df_train_predict = pd.DataFrame(list(zip(X_predict, y_train_pred)), columns = ['object_id', 'pred_likes'])

In [ ]:
df_train_predict

,object_id,pred_likes
0,0011d6be41612ec9eae3,30.471176
1,0012765f7a97ccc3e9e9,1.910035
2,00181d86ff1a7b95864e,82.239296
3,001c52ae28ec106d9cd5,84.744156
4,001f4c71b4d53497b531,1.595925
...,...,...
12002,ffedf8af4fd5b3873164,3.066441
12003,ffee34705ea44e1a0f79,1.455457
12004,ffefbe1faf771aa4f790,1.833020
12005,fff08e76cbb969eaddc7,2.040512


In [ ]:
df_test_predict = pd.DataFrame(list(zip(X_test_predict, y_test_pred)), columns = ['object_id', 'pred_likes'])

In [ ]:
df_test_predict

,object_id,pred_likes
0,000405d9a5e3f49fc49d,0.893163
1,001020bd00b149970f78,61.201641
2,00133be3ff222c9b74b0,2.697483
3,001b2b8c9d3aa1534dfe,1.954475
4,00220cd4bfa082d2aa20,1.581104
...,...,...
11983,fff4bbb55fd7702d294e,1.654351
11984,fffbe07b997bec00e203,1.277655
11985,fffd1675758205748d7f,1.753125
11986,fffd43b134ba7197d890,1.828647


In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/second_take


In [ ]:
df_train_predict.to_pickle('./model_temp/cnn_train_predict.pkl')
df_test_predict.to_pickle('./model_temp/cnn_test_predict.pkl')

In [ ]:
Y = []
unique_train_ids = train["object_id"].unique()
unique_test_ids = test["object_id"].unique()
#N_IMAGES_FOR_OBJ_ID = 1
N_OBJ_IDS = len(unique_train_ids)
unique_palette_obj = new_palette["object_id"].unique()

for i, like in tqdm(zip(range(N_OBJ_IDS), train['likes'])):
  obj_id = unique_train_ids[i]
  if obj_id not in unique_palette_obj:
      continue
  Y.append(like)
#Y = np.ravel(Y)


12026it [00:06, 1910.22it/s]


In [ ]:
get_score(y_train_pred, Y)

1.3279886774229985

# 画像の表示など

In [ ]:
# unique_object_ids = new_palette["object_id"].unique()
# N_IMAGES_FOR_OBJ_ID = 6
# N_OBJ_IDS = 5

# fig, axes = plt.subplots(nrows=N_OBJ_IDS, ncols=N_IMAGES_FOR_OBJ_ID, figsize=(25, 25))
# for i in range(N_OBJ_IDS):
#     obj_id = unique_object_ids[i]
#     palette_obj_id = new_palette.query(f"object_id == '{obj_id}'")
#     axes[i, 0].set_ylabel(obj_id)
#     for j in range(N_IMAGES_FOR_OBJ_ID):
#         generated = _create_random_image(palette_obj_id)
#         axes[i, j].imshow(generated)
#         axes[i, j].grid(False)
#         axes[i, j].set_title(f"Generated Image {j}")
        
# plt.tight_layout()
# plt.show()

こうしてみてみると同じ`object_id`から生成された画像はパターンこそ違えど似ていて、異なる`object_id`どうしでははっきりと見分けることができます。この観察をうまく活かして、`palette.csv`をなんとか固定長の特徴ベクトル表現にしたい!という時に自己教師あり対照学習の利用を思いつきました。

## 自己教師あり対照学習

自己教師あり対照学習についてサクッと説明します。と言っても私も詳しくないのであまり大したことはお話しできません。まず自己教師あり学習についてですが、「入力データの一部に機械的に変換を施しその変換に対し不変の(invariant)な表現を学習する教師なし学習の１手法」です(間違っているかもしれません)。例えば自然言語処理をはじめとして近年世間を騒がせているBERTやその派生も「入力データの一部をマスクしてその部分を予測させる」という自己教師あり学習を行っています(Word2Vecもそうですね)。

自己教師あり対照学習は、自己教師あり学習に含まれる一つの方法論で、入力データに変換を施した上で特徴表現を比較するようにして学習を行う手法です。例えば2020年に提案された[SimCLR](https://arxiv.org/abs/2002.05709)は画像に変換(Data Augmentation)を施し、同じ画像から由来する異なるData Augmentationがかけられた画像特徴を近づけるようにしつつ、異なる画像に由来する画像特徴から特徴空間上で反発するような制約を課すことで良い画像特徴を学習することを目指した手法です。

![SimCLR](https://webbigdata.jp/wp-content/uploads/2020/04/illustration-of-the-proposed-SimCLR-framework.gif)

## `palette`に関する表現を学習する

さて、この自己教師あり対照学習の考え方で、`palette`に関する表現を学習する方法を考えてみましょう。やり方の大枠は同じで、同じ`object_id`から生成された二つのランダム配置の画像特徴が似るように、異なる`object_id`から生成されたランダム配置の画像とは画像特徴が特徴空間上で離れるようなロスを設計してやれば良いはずです。

![How-to-embed-palette](https://gist.githubusercontent.com/koukyo1994/072f7feb3c966cf91fb672006b6d0dd6/raw/c20b02e46d8f3d4c87bcae06b76d9de875f1168f/ColorEmbedding.png)

このアイデアをPyTorchで実装してみます。

### Datasetの定義

アンカー画像、正例、負例をそれぞれ作成するデータセットを作成します。

In [ ]:
class ColorImageDataset(torchdata.Dataset):
    def __init__(self, df: pd.DataFrame, transforms=None):
        self.object_id = df["object_id"].unique()
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.object_id)
    
    def __getitem__(self, idx: int):
        object_id = self.object_id[idx]
        sample = self.df.query(f"object_id == '{object_id}'")[
            ["ratio_int", "color_r", "color_g", "color_b"]]
        # 負例のサンプリングを行う
        while True:
            neg_sample_id = np.random.choice(self.object_id)
            if neg_sample_id != object_id:
                break
        neg_sample = self.df.query(f"object_id == '{neg_sample_id}'")[
            ["ratio_int", "color_r", "color_g", "color_b"]]
        
        # アンカー画像の生成
        anchor = _create_random_image(sample)
        # 正例の生成
        pos = _create_random_image(sample)
        # 負例の生成
        neg = _create_random_image(neg_sample)
        
        anchor = self.transforms(image=anchor)["image"]
        pos = self.transforms(image=pos)["image"]
        neg = self.transforms(image=neg)["image"]
        return anchor, pos, neg

### CNNモデルの定義

2層のCNNで学習をおこないます。出力は64次元のベクトルになります。

In [ ]:
# class CNNModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.cnn_encoder = nn.Sequential(
#             nn.Conv2d(3, 32, 3),
#             nn.ReLU(),
#             nn.Conv2d(32, 64, 3),
#             nn.ReLU())

#     def forward(self, x):
#         return self.cnn_encoder(x).mean(dim=[2, 3])

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn_encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.Sigmoid(),
            nn.Conv2d(32, 64, 3),
            nn.Sigmoid())

    def forward(self, x):
        return self.cnn_encoder(x).mean(dim=[2, 3])

### 損失関数の定義

対照学習ではさまざまな損失関数が提案されているようなのですが、一旦適当な損失関数として、アンカー画像と正例のコサイン類似度を大きくしつつ、アンカー画像と負例のコサイン類似度は小さくなるような学習をおこなうことにします。

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cos = nn.CosineSimilarity()
        
    def forward(self, anchor, pos, neg):
        pos_loss = 1.0 - self.cos(anchor, pos).mean(dim=0)
        neg_loss = self.cos(anchor, neg).mean(dim=0)
        return pos_loss + neg_loss

### その他学習用の用意

Catalystを用いて学習を行うための準備をします。

In [ ]:
class ContrastRunner(Runner):
    def predict_batch(self, batch, **kwargs):
        return super().predict_batch(batch, **kwargs)
    
    def handle_batch(self, batch):
        anchor, pos, neg = batch[0], batch[1], batch[2]
        anchor = anchor.to(self.device)
        pos = pos.to(self.device)
        neg = neg.to(self.device)
        
        anchor_emb = self.model(anchor)
        pos_emb = self.model(pos)
        neg_emb = self.model(neg)
        
        loss = self.criterion(anchor_emb, pos_emb, neg_emb)
        self.batch_metrics.update({
            "loss": loss
        })
        
        self.input = batch
        if self.is_train_loader:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

In [ ]:
class SchedulerCallback(Callback):
    def __init__(self):
        super().__init__(CallbackOrder.Scheduler)

    def on_loader_end(self, state: IRunner):
        lr = state.scheduler.get_last_lr()
        state.epoch_metrics["lr"] = lr[0]
        if state.is_train_loader:
            state.scheduler.step()

In [ ]:
def set_seed(seed=1996):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
OUTDIR = Path("../output/PaletteEmbedding")
OUTDIR.mkdir(exist_ok=True, parents=True)

## 学習のループ

普通のKFoldで行います。今回は`likes`の情報を用いないためtest側に属している`object_id`も学習に用いることができます。

In [ ]:
# MODEL_DIR = "./model_temp"

# if not os.path.exists(MODEL_DIR):  # ディレクトリが存在しない場合、作成する。
#     os.makedirs(MODEL_DIR)
# checkpoint = ModelCheckpoint(
#     filepath=os.path.join(MODEL_DIR, "model-{epoch:02d}.h5"), save_best_only=True) 

In [ ]:
# #kf = KFold(n_splits=2, random_state=1996, shuffle=True)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# set_seed(1996)




# unique_obj_id = new_palette["object_id"].unique()
# print("*" * 100)
# #print(f"Fold: {fold}")
# print(trn_idx)
# print(val_idx)
# print(len(trn_idx))
# print(len(val_idx))

In [ ]:
# kf = KFold(n_splits=2, random_state=1996, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(1996)


unique_obj_id = new_palette["object_id"].unique()

permutation = np.random.permutation(len(unique_obj_id))
trn_idx = sorted(permutation[len(unique_obj_id)//2:])
val_idx = sorted(permutation[:len(unique_obj_id)//2])


#for fold, (trn_idx, val_idx) in enumerate(kf.split(unique_obj_id)):
print("*" * 100)
#print(f"Fold: {fold}")

trn_obj_id = unique_obj_id[trn_idx]
val_obj_id = unique_obj_id[val_idx]


trn_palette = new_palette[
    new_palette["object_id"].isin(trn_obj_id)
].reset_index(drop=True)
val_palette = new_palette[
    new_palette["object_id"].isin(val_obj_id)
].reset_index(drop=True)

transforms = A.Compose([A.Normalize(), ToTensorV2()])
trn_dataset = ColorImageDataset(trn_palette, transforms)
val_dataset = ColorImageDataset(val_palette, transforms)

trn_loader = torchdata.DataLoader(
    trn_dataset, batch_size=128, shuffle=True, num_workers=20)
val_loader = torchdata.DataLoader(
    val_dataset, batch_size=256, shuffle=False, num_workers=20)

model = CNNModel().to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
callbacks = [SchedulerCallback()]
#callbacks = [checkpoint]
runner = ContrastRunner(engine = device)
runner.train(model=model,
              criterion=criterion,
              optimizer=optimizer,
              scheduler=scheduler,
              callbacks=callbacks,
              loaders={"train": trn_loader, "valid": val_loader},
              num_epochs=20,
              # logdir=OUTDIR,
              # verbose=True
             )

****************************************************************************************************
Hparams (experiment): {}


## 学習された特徴表現を得る

さて、学習が済んだので今度は学習された特徴表現ベクトルを`object_id`ごとに得ます。

In [ ]:
#kf = KFold(n_splits=5, random_state=1213, shuffle=True)
embeddings = []
object_ids = []
#for fold, (_, val_idx) in enumerate(kf.split(unique_obj_id)):
fold = 0
trn_obj_id = unique_obj_id[trn_idx]
trn_palette = new_palette[
    new_palette["object_id"].isin(trn_obj_id)
].reset_index(drop=True)
trn_dataset = ColorImageDataset(trn_palette, transforms)
object_ids.extend(trn_dataset.object_id.tolist())

trn_loader = torchdata.DataLoader(trn_dataset, batch_size=256, shuffle=False, num_workers=20)
model = CNNModel()
ckpt = torch.load(OUTDIR / f"fold{fold}/checkpoints/best.pth")
model.load_state_dict(ckpt["model_state_dict"])
model.to(device)
model.eval()
# アンカー画像にのみ推論
for anchor, _, _ in tqdm(trn_loader):
    anchor = anchor.to(device)
    with torch.no_grad():
        embedding = model(anchor).detach().cpu().numpy()
    embeddings.append(embedding)

In [ ]:
all_embeddings = np.concatenate(embeddings, axis=0)
len(all_embeddings), len(object_ids)

In [ ]:
embedding_df_train = pd.DataFrame(all_embeddings, 
                            columns=[f"color_embedding_{i}" for i in range(len(all_embeddings[0]))],
                            index=object_ids)
embedding_df_train#.head()

In [ ]:
#kf = KFold(n_splits=5, random_state=1213, shuffle=True)
embeddings = []
object_ids = []
#for fold, (_, val_idx) in enumerate(kf.split(unique_obj_id)):
fold = 0
val_obj_id = unique_obj_id[val_idx]
val_palette = new_palette[
    new_palette["object_id"].isin(val_obj_id)
].reset_index(drop=True)
val_dataset = ColorImageDataset(val_palette, transforms)
object_ids.extend(val_dataset.object_id.tolist())

val_loader = torchdata.DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=20)
model = CNNModel()
ckpt = torch.load(OUTDIR / f"fold{fold}/checkpoints/best.pth")
model.load_state_dict(ckpt["model_state_dict"])
model.to(device)
model.eval()
# アンカー画像にのみ推論
for anchor, _, _ in tqdm(val_loader):
    anchor = anchor.to(device)
    with torch.no_grad():
        embedding = model(anchor).detach().cpu().numpy()
    embeddings.append(embedding)

In [ ]:
all_embeddings = np.concatenate(embeddings, axis=0)
len(all_embeddings), len(object_ids)

In [ ]:
embedding_df_valid = pd.DataFrame(all_embeddings, 
                            columns=[f"color_embedding_{i}" for i in range(len(all_embeddings[0]))],
                            index=object_ids)
embedding_df_valid#.head()

In [ ]:
embedding_df = pd.concat([embedding_df_train, embedding_df_valid])
embedding_df

In [ ]:
embedding_df.to_pickle('./model_temp/palette_embedding999.pkl')

In [ ]:
palette[['object_id']].nunique()

特徴表現が得られていることがわかります。

## UMAPで圧縮しlikesと相関がありそうかみてみる

得られた特徴表現が役立ちそうかみてみましょう。

In [ ]:
reducer = umap.UMAP(random_state=42)
reduced = reducer.fit_transform(embedding_df.values)
umap_df = pd.DataFrame(reduced, columns=["dim0", "dim1"], index=embedding_df.index)
umap_df.head()

In [ ]:
train = pd.read_csv(DATADIR / "train.csv")
train["likes"] = np.log1p(train["likes"])
likes_df = train[["object_id", "likes"]]
likes_df.head()

In [ ]:
likes_df = likes_df.merge(umap_df, left_on="object_id", right_index=True, how="left")
likes_df

In [ ]:
plt.figure(figsize=(10, 10))
sns.scatterplot(x="dim0", y="dim1", hue="likes", data=likes_df, alpha=0.5);

どうやら`likes`が多いサンプルは一部に集まっているようです。特徴として使えるかもしれません。

In [ ]:
# 得られた表現を保存する
# embedding_df.reset_index(),rename(columns={"index": "object_id"}).to_csv("../input/palette_embedding.csv", index=False)

## 議論と考察

ここでは上の実験に関していくつか改善できる点をあげたり、どのような学習がなされていそうかといった考察を行います。

まず、上の実装に関してですがいくつか問題があります(私が実際サブミットに使ったものと一貫性を取りたかったためあえてそのままにしています)。

* 上の実装ではKFoldでFoldを切って学習をしているがこれをやらない方がいい可能性がある

わざわざKFoldを切って学習をしているのですが、これはうっかり惰性でやってしまっただけで本来必要はありません。というかおそらくやってしまうとあまり良くありません。なぜかというと、各foldで学習されたモデルはそれぞれ**違う特徴空間への射影を学習している**ため、後で特徴として利用しようとするときには5つの異なる特徴空間を無理やりくっつけたような特徴空間になってしまうからです。当然異なる特徴空間どうしでは近いか遠いかを区別できないため問題が生じます。解決策としてはKFoldを切らず、全データを用いて学習をする、ということが挙げられます。今回は特にターゲットの値を使って学習をしているわけではないのでリークの心配はありません。

* 最終層がReLU()をかけた出力になっている

得られた特徴表現が非常にスパースになっていることに気づいた方も多いかと思いますが、これは私がうっかりReLU()の出力を出してしまっていて負値を全て0にしてしまっているからです。これもうっかりミスでやってしまっているので直した方がいいかもしれません。

以上の2点が実装上のミスで出てしまっている問題点のため、直すことで改善が見込めるかもしれません。


また、上記の学習プロセスで何を学習させているのか、ということを考察すると、改善の余地が見つかるかもしれません。損失関数に関して一つ考察を述べておこうと思います。まず、Anchorと正例の間のコサイン類似度を大きくするように学習する点は、色の配置に関する不変性を学習させていることに相当します(permutation invariance)。つまり色を表すタイルの配置に意味はない、という点を明示的に損失として与えています。一方、Anchorと負例の間のコサイン類似度を小さくする損失は異なるソースからでたデータをが特徴空間上で異なるような制約になっていますが、この制約は例えば色の比率やコントラストなどに関して明示的には制約をかけていないため、ひょっとすると平均色の違いを学習しているだけになっている可能性もあります。このようなことを考えると以下のような改善法があり得るかもしれません。

* 損失関数の変更

今回は単純にコサイン類似度のみを用いていますが、この部分に関してどのあたりに注目して欲しいかという気持ちを込めて変更できるといいかもしれません。

* Data Augmentationの適用

今回は正例としてAnchorと配置が違うだけの画像、負例としてAnchorと異なる画像を用いていますが、例えば負例としてAnchorの画像の色に関して変動を加えた画像を用いる、なども考えられます。

さらに、今回はあえて画像の入力として2D CNNを適用してみましたが、そもそも色の比率のみが問題とすると実は1D(色の3ch分を数えると2D)の点列として考えて同じような学習を行う、なども改善案としてはあります。この場合には2DCNNではなく、1DCNNやTransformerなどを用いることが考えられます。

いずれにせよ今回の実験はかなり改善の余地があるため、興味ドリブンでやってみたよ、くらいのノリだと思ってください。

## EOF